In [1]:
import matplotlib.pyplot as plt
import numpy as np
import joblib as jb
import helper_functions as hlf

%matplotlib inline

Using TensorFlow backend.


In [2]:
# load preprocessed dragonfly images
trainpath = r'D:\Linnaeus_models\dragon\train\dragon_train.npy'
testpath = r'D:\Linnaeus_models\dragon\test\dragon_test.npy'
dragons = np.concatenate((np.load(trainpath), np.load(testpath)), axis=0)
print(f'Number of images: {len(dragons)}')

Number of images: 18407


##  Image Denoising with CNN
   * Test image denoising abilities on the images

In [3]:
# separate into training set and validation set and reshape to fit the NN
x_train = dragons[0:16000] 
x_val = dragons[16000:]

In [4]:
x_train = np.expand_dims(x_train, axis=-1)
x_val = np.expand_dims(x_val, axis=-1)

In [5]:
model = hlf.create_unsupervised_model()

### Begin training

In [ ]:
# train model
train_history = model.fit(x_train, x_train, epochs=8, batch_size=100, 
                          validation_data=(x_val, x_val))
# save model
model_path =  r'D:\Linnaeus_models\dragon_reconstruction_v1.pkl'
jb.dump(loaded_model, model_path)

### Reload unsupervised model to continue training

In [6]:
# load the model
loaded_model = jb.load(r'D:\Linnaeus_models\dragon_reconstruction_v4.pkl')

In [ ]:
# continue training the unsupervised model
train_history = loaded_model.fit(x_train, x_train, epochs=2, batch_size=100, 
                                 validation_data=(x_val, x_val))

# save model and weights
loaded_model.save_weights(r'D:\Linnaeus_models\dragon_reconstruction_v4_weights')
model_path =  r'D:\Linnaeus_models\dragon_reconstruction_v4.pkl'
jb.dump(loaded_model, model_path)

Train on 16000 samples, validate on 2407 samples
Epoch 1/2
15300/16000 [===========================>..] - ETA: 1:00:10 - loss: 0.00 - ETA: 55:32 - loss: 0.0087 - ETA: 53:10 - loss: 0.00 - ETA: 51:36 - loss: 0.00 - ETA: 51:05 - loss: 0.00 - ETA: 50:50 - loss: 0.00 - ETA: 50:09 - loss: 0.00 - ETA: 49:34 - loss: 0.00 - ETA: 49:02 - loss: 0.00 - ETA: 48:33 - loss: 0.00 - ETA: 48:08 - loss: 0.00 - ETA: 47:48 - loss: 0.00 - ETA: 47:23 - loss: 0.00 - ETA: 47:22 - loss: 0.00 - ETA: 47:36 - loss: 0.00 - ETA: 48:01 - loss: 0.00 - ETA: 48:21 - loss: 0.00 - ETA: 48:12 - loss: 0.00 - ETA: 47:42 - loss: 0.00 - ETA: 47:15 - loss: 0.00 - ETA: 46:46 - loss: 0.00 - ETA: 46:18 - loss: 0.00 - ETA: 45:52 - loss: 0.00 - ETA: 45:25 - loss: 0.00 - ETA: 44:59 - loss: 0.00 - ETA: 44:32 - loss: 0.00 - ETA: 44:07 - loss: 0.00 - ETA: 43:42 - loss: 0.00 - ETA: 43:17 - loss: 0.00 - ETA: 42:58 - loss: 0.00 - ETA: 42:39 - loss: 0.00 - ETA: 42:17 - loss: 0.00 - ETA: 41:56 - loss: 0.00 - ETA: 41:34 - loss: 0.00 - ETA: 4

In [ ]:
preds = loaded_model.predict(x_val)
jb.dump(preds, '.test_images/unsupervised_val_images.pkl')

In [ ]:
preds = jb.load('./unsupervised_val_images.pkl')

In [ ]:
reverted_x = np.squeeze(x_val, axis=-1)